# Imports

In [1]:
import sys
sys.path.insert(0, '/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/mayas_project/hgru_model/model/')

In [2]:
import pandas as pd
import numpy as np
import pickle
import itertools
import random
import torch
import optuna
from model.utils import *

/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-11-11 11:37:08.453398: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
from pipeline_config import *

# Seeds for Comparisons:

In [4]:
torch.manual_seed(1)
np.random.seed(2)
random.seed(3)

In [5]:
Year

2020

In [6]:
#pip install "pandas<2.0.0"

# Read Data

In [6]:
with open(son_parent_path, 'rb') as f:
    son_parent_dict = pickle.load(f)

with open(son_parent_corr_path, 'rb') as f:
    son_parent_corr_dict = pickle.load(f)

with open(train_dataset_dict_path, 'rb') as f:
    train_dataset_dict = pickle.load(f)

with open(test_dataset_dict_path, 'rb') as f:
    test_dataset_dict = pickle.load(f)

with open(category_id_to_category_name_path, 'rb') as f:
    category_id_to_name_dict = pickle.load(f)
    
with open(categories_per_indent_path, 'rb') as f:
    categories_per_indent_dict = pickle.load(f)

In [7]:
len(train_dataset_dict.keys())

350

In [8]:
len(category_id_to_name_dict.keys())

350

# Hierarchical GRU

In [9]:
def objective(trial):
    with open(son_parent_path, 'rb') as f:
        son_parent_dict = pickle.load(f)
    
    with open(son_parent_corr_path, 'rb') as f:
        son_parent_corr_dict = pickle.load(f)

    with open(train_dataset_dict_path, 'rb') as f:
        train_dataset_dict = pickle.load(f)

    with open(test_dataset_dict_path, 'rb') as f:
        test_dataset_dict = pickle.load(f)

    with open(category_id_to_category_name_path, 'rb') as f:
        category_id_to_name_dict = pickle.load(f)
        
    with open(categories_per_indent_path, 'rb') as f:
        categories_per_indent_dict = pickle.load(f)

    weights_path = weightspath
    
    #--------------------------------------------------------------------------------------------------------------------------------------#

    loss_coef = trial.suggest_float('loss_coef_1',  1e-10, 1e-1, log=True)
    Lr = trial.suggest_float('Lr', 1e-5, 1e-1, log=True)

    hgru_models = {}
    num_categories = 0

    for indent in sorted(list(categories_per_indent_dict.keys())):
    #for indent in [1,2]:
        for category in categories_per_indent_dict[indent]:
            num_categories +=1
            print(f'num categories: {num_categories}')
            category_name = category_id_to_name_dict[category]
            print(f'category id|name: {category}|{category_name}')

            if int(indent) == 0 or son_parent_dict[category] not in categories_per_indent_dict[indent-1] or (category, son_parent_dict[category]) not in son_parent_corr_dict.keys():
                loss_coef=0
                parent_weights=0
            else:
                son = category
                parent = son_parent_dict[son]
                parent_name = category_id_to_name_dict[parent]
                parent_model = GRUModel(input_dim=Features, hidden_dim=HiddenSize, layer_dim=LayersDim, output_dim=OutputDim, dropout_prob=DropoutProb)
                parent_optimizer = torch.optim.AdamW(parent_model.parameters(), lr=Lr)
                parent_model, optimizer, checkpoint, valid_loss_min = load_checkpoint(weights_path+parent_name+'.pt', parent_model, parent_optimizer)
                parent_weights = unify_model_weights(parent_model)

            train_dataloader, test_dataloader = create_dataloader(train_dataset_dict[category_name], test_dataset_dict[category_name])
            model = GRUModel(input_dim=Features, hidden_dim=HiddenSize, layer_dim=LayersDim, output_dim=OutputDim, dropout_prob=DropoutProb)
            optimizer = torch.optim.AdamW(model.parameters(), lr=Lr)
            model.to(Device)
            saving_param_path = weights_path+category_name+'.pt'
            min_error = training_and_evaluation(trial, model, train_dataloader, test_dataloader, optimizer, category_name, parent_weights, loss_coef, path=saving_param_path)
            hgru_models[category] = min_error

    average_error = sum(list(hgru_models.values()))/len(list(hgru_models.values()))
    return average_error


In [10]:
EPOCHS = 30
    
study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler())
study.optimize(objective, n_trials=30)

[I 2023-11-11 11:38:27,144] A new study created in memory with name: no-name-1dacba5f-fd1d-4ef3-9af1-42d9db357bc3


num categories: 1
category id|name: 8106.0|All items
num categories: 2
category id|name: 4604.0|All items less energy


/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/.venv/lib/python3.11/site-packages/optuna/trial/_trial.py:490: UserWarning: The reported value is ignored because this `step` 0 is already reported.
  warnings.warn(
/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/.venv/lib/python3.11/site-packages/optuna/trial/_trial.py:490: UserWarning: The reported value is ignored because this `step` 1 is already reported.
  warnings.warn(
/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/.venv/lib/python3.11/site-packages/optuna/trial/_trial.py:490: UserWarning: The reported value is ignored because this `step` 2 is already reported.
  warnings.warn(
/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/.venv/lib/python3.11/site-packages/optuna/trial/_trial.py:490: UserWarning: The reported value is ignored because this `step` 3 is already reported.
  warnings.warn(
/Users/mvilenko/Library/CloudSto

num categories: 3
category id|name: 3531.0|All items less food
num categories: 4
category id|name: 5306.0|All items less food and energy
num categories: 5
category id|name: 4114.0|All items less food and shelter
num categories: 6
category id|name: 5789.0|All items less food, shelter, and energy
num categories: 7
category id|name: 2208.0|All items less food, shelter, energy, and used cars and trucks
num categories: 8
category id|name: 2047.0|All items less medical care
num categories: 9
category id|name: 7199.0|All items less shelter
num categories: 10
category id|name: 3242.0|Apparel less footwear
num categories: 11
category id|name: 704.0|Commodities
num categories: 12
category id|name: 2665.0|Durables
num categories: 13
category id|name: 1868.0|Education and communication
num categories: 14
category id|name: 7558.0|Energy
num categories: 15
category id|name: 4934.0|Food
num categories: 16
category id|name: 9257.0|Food and beverages
num categories: 17
category id|name: 5463.0|Fuels an

[I 2023-11-11 13:04:43,604] Trial 0 finished with value: 0.09503653014602605 and parameters: {'loss_coef_1': 7.373705905658308e-07, 'Lr': 0.008958748095198659}. Best is trial 0 with value: 0.09503653014602605.


...

[I 2023-11-12 17:09:41,631] Trial 20 finished with value: 0.09505004320222984 and parameters: {'loss_coef_1': 2.3390196757768757e-05, 'Lr': 0.008017025170339604}. Best is trial 12 with value: 0.07937231612466608.


num categories: 1
category id|name: 8106.0|All items
num categories: 2
category id|name: 4604.0|All items less energy


/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/.venv/lib/python3.11/site-packages/optuna/trial/_trial.py:490: UserWarning: The reported value is ignored because this `step` 0 is already reported.
  warnings.warn(
/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/.venv/lib/python3.11/site-packages/optuna/trial/_trial.py:490: UserWarning: The reported value is ignored because this `step` 1 is already reported.
  warnings.warn(
/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/.venv/lib/python3.11/site-packages/optuna/trial/_trial.py:490: UserWarning: The reported value is ignored because this `step` 2 is already reported.
  warnings.warn(
/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/.venv/lib/python3.11/site-packages/optuna/trial/_trial.py:490: UserWarning: The reported value is ignored because this `step` 3 is already reported.
  warnings.warn(
/Users/mvilenko/Library/CloudSto

num categories: 3
category id|name: 3531.0|All items less food
num categories: 4
category id|name: 5306.0|All items less food and energy
num categories: 5
category id|name: 4114.0|All items less food and shelter
num categories: 6
category id|name: 5789.0|All items less food, shelter, and energy
num categories: 7
category id|name: 2208.0|All items less food, shelter, energy, and used cars and trucks
num categories: 8
category id|name: 2047.0|All items less medical care
num categories: 9
category id|name: 7199.0|All items less shelter
num categories: 10
category id|name: 3242.0|Apparel less footwear
num categories: 11
category id|name: 704.0|Commodities
num categories: 12
category id|name: 2665.0|Durables
num categories: 13
category id|name: 1868.0|Education and communication
num categories: 14
category id|name: 7558.0|Energy
num categories: 15
category id|name: 4934.0|Food


: 

In [11]:
best_trial = study.best_trial
best_trial

NameError: name 'study' is not defined

In [ ]:
def get_results_on_test_set(weights_path, train_dataset_dict, test_dataset_dict, categories = None):
    predictions_dict = {}
    if categories is None:
        categories = list(test_dataset_dict.keys())
        
    for category in categories:
        print(category)
        train_dataloader, test_dataloader = create_dataloader(train_dataset_dict[category], test_dataset_dict[category])
        basic_model = Model
        basic_optimizer = Optimizer
        ckp_path = weights_path+category+'.pt'
        model, optimizer, checkpoint, valid_loss_min = load_checkpoint(ckp_path, basic_model, basic_optimizer)
        predictions = get_predictions_on_test_set(model, test_dataloader)
        predictions_dict[category] = predictions
    return predictions_dict

In [ ]:
categories_lists = list(categories_per_indent_dict.values())
categories_id = list(itertools.chain.from_iterable(categories_lists))
categories = []
for category_id in categories_id:
    categories.append(category_id_to_name_dict[category_id])

In [ ]:
predictions_dict = get_results_on_test_set(weightspath, train_dataset_dict, test_dataset_dict, categories = categories)

In [ ]:
with open(test_predictions_path, 'wb') as handle:
    pickle.dump(predictions_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Get Best Model Weights:

In [ ]:
#def get_weights_per_category(category_list, dir_path):
#    weights = {}
#    for category in category_list:
#        model = GRUModel(input_dim = Features, hidden_dim = HiddenSize, layer_dim = LayersDim, output_dim = OutputDim, dropout_prob = DropoutProb)
#        model.to(device)
#        optimizer = torch.optim.AdamW(model.parameters(), lr=Lr)
        
#        best_checkpoint_path = dir_path+category + '.pt'

#        category_model, optimizer, checkpoint, valid_loss_min = load_checkpoint(best_checkpoint_path, model, optimizer)
#        category_model_weights = unify_model_weights(category_model)

#        weights[category] = category_model_weights
    
#    return weights

In [ ]:
def get_weights_per_category(category_id_list, dir_path):
    basic_model = GRUModel(input_dim = Features, hidden_dim = HiddenSize, layer_dim = LayersDim, output_dim = OutputDim, dropout_prob = DropoutProb)
    basic_optimizer = torch.optim.AdamW(basic_model.parameters(), lr=Lr)
    #basic_model.to(device)

    best_models_weights_dict = {}

    for category_id in category_id_list:
        category_name = category_id_to_name_dict[category_id]
        ckp_path = dir_path+category_name+'.pt'
        model, optimizer, checkpoint, valid_loss_min = load_checkpoint(ckp_path, basic_model, basic_optimizer)
        category_model_weights = unify_model_weights(model)
        best_models_weights_dict[category_id] = category_model_weights
        
    return best_models_weights_dict

In [ ]:
dir_path = '/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/mayas_project/hgru_model/models_weights/'

In [ ]:
category_id_list = []
 
# list out keys and values separately
key_list = list(category_id_to_name_dict.keys())
val_list = list(category_id_to_name_dict.values())

for cat_name in categories:
    position = val_list.index(cat_name)
    category_id_list.append(key_list[position])


In [ ]:
len(categories)

In [ ]:
len(category_id_list)

In [ ]:
weights_dict = get_weights_per_category(category_id_list, dir_path)

In [ ]:
with open('/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/CPI_HRNN - version 2.0/pickle files/hgru_model_weights.pickle', 'wb') as handle:
    pickle.dump(weights_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)